In [ ]:
import json
import os
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Union

import fitz
import numpy as np
from langchain_ollama import ChatOllama, OllamaEmbeddings
from numpy.linalg import norm

In [ ]:
class Document:
    """Class to represent a document with its content and metadata."""

    def __init__(self, content: str, metadata: Optional[Dict] = None):
        """
        Initialize a document with content and optional metadata.

        Args:
            content (str): The text content of the document
            metadata (Optional[Dict]): Additional information about the document (e.g., source, author)
        """
        self.content = content
        self.metadata = metadata or {}

    def to_dict(self) -> Dict:
        """Convert the document to a dictionary for serialization."""
        return {"content": self.content, "metadata": self.metadata}

    @classmethod
    def from_dict(cls, data: Dict) -> "Document":
        """Create a Document from a dictionary."""
        return cls(content=data["content"], metadata=data.get("metadata", {}))

In [ ]:
class VectorStore:
    """Simple in-memory vector store implementation for demonstration purposes."""

    def __init__(self):
        """Initialize the vector store with empty storage structures."""
        self.documents: List[Document] = []
        self.embeddings: List[np.ndarray] = []
        self.embedding_model = OllamaEmbeddings(model="llama3.2:3b")

    def add_document(self, document: Document) -> None:
        """
        Add a document to the vector store and generate its embedding.

        Args:
            document (Document): The Document object to add
        """
        try:
            # Generate embedding for the document content
            embedding = self.embedding_model.embed_documents([document.content])[0]
            embedding_array = np.array(embedding)

            # Store document and its embedding
            self.documents.append(document)
            self.embeddings.append(embedding_array)
        except Exception as e:
            print(f"Error adding document: {e}")
            raise

    def similarity_search(self, query: str, k: int = 3) -> List[Tuple[Document, float]]:
        """
        Find the k most similar documents to the query.

        Args:
            query (str): The search query text
            k (int): Number of similar documents to return (default: 3)

        Returns:
            List of tuples containing (document, similarity_score)
        """
        try:
            # Generate embedding for the query
            query_embedding = np.array(self.embedding_model.embed_documents([query])[0])

            # Calculate cosine similarities
            similarities = []
            for doc_embedding in self.embeddings:
                cosine_sim = np.dot(query_embedding, doc_embedding) / (
                    norm(query_embedding) * norm(doc_embedding)
                )
                similarities.append(cosine_sim)

            # Get indices of top k similar documents
            top_k_indices = np.argsort(similarities)[-k:][::-1]

            # Return documents with their similarity scores
            return [(self.documents[i], similarities[i]) for i in top_k_indices]
        except Exception as e:
            print(f"Error in similarity search: {e}")
            raise

    def save_to_disk(self, filepath: str) -> None:
        """Save the vector store to disk."""
        try:
            data = {
                "documents": [doc.to_dict() for doc in self.documents],
                "embeddings": [embedding.tolist() for embedding in self.embeddings],
            }
            with open(filepath, "w") as f:
                json.dump(data, f)
        except Exception as e:
            print(f"Error saving vector store: {e}")
            raise

    @classmethod
    def load_from_disk(cls, filepath: str) -> "VectorStore":
        """Load a vector store from disk."""
        try:
            with open(filepath, "r") as f:
                data = json.load(f)

            store = cls()
            store.documents = [Document.from_dict(doc) for doc in data["documents"]]
            store.embeddings = [np.array(embedding) for embedding in data["embeddings"]]
            return store
        except Exception as e:
            print(f"Error loading vector store: {e}")
            raise

In [ ]:
class RAGSystem:
    """Retrieval-Augmented Generation system with feedback loop implementation."""

    def __init__(self, llm_model: str = "llama3.2:3b"):
        """
        Initialize the RAG system.

        Args:
            llm_model (str): Name of the Ollama model to use
        """
        self.vector_store = VectorStore()
        self.llm = ChatOllama(model=llm_model)
        self.feedback_log = []

    def ingest_pdf_documents(self, pdf_paths: List[str]) -> None:
        """
        Ingest multiple PDF documents into the vector store.

        Args:
            pdf_paths (List[str]): List of paths to PDF files
        """
        for pdf_path in pdf_paths:
            try:
                print(f"Processing PDF: {pdf_path}")
                # Extract text from PDF using Fitz
                doc = fitz.open(pdf_path)
                full_text = ""
                for page in doc:
                    full_text += page.get_text()

                # Create document with filename as metadata
                document = Document(
                    content=full_text, metadata={"source": os.path.basename(pdf_path)}
                )

                # Add to vector store
                self.vector_store.add_document(document)
                print(f"Successfully processed: {pdf_path}")
            except Exception as e:
                print(f"Error processing {pdf_path}: {e}")
                continue

    def generate_response(self, query: str, k: int = 3) -> str:
        """
        Generate a response to a query using RAG approach.

        Args:
            query (str): The user's question or input
            k (int): Number of relevant documents to retrieve (default: 3)

        Returns:
            Generated response from the LLM
        """
        try:
            # Retrieve relevant documents
            relevant_docs = self.vector_store.similarity_search(query, k=k)

            # Format the context from retrieved documents
            context = "\n\n".join(
                [
                    f"Source {i + 1} (from {doc.metadata.get('source', 'unknown')}):\n{doc.content[:1000]}..."
                    for i, (doc, _) in enumerate(relevant_docs)
                ]
            )

            # Create the prompt with context
            prompt = f"""You are a helpful AI assistant. Use the following context to answer the question at the end.
            
            Context:
            {context}
            
            Question: {query}
            
            Provide a comprehensive answer based on the context. If the context doesn't contain enough information,
            say "I don't have enough information to answer this question properly."
            
            Answer:"""

            # Generate response from LLM
            response = self.llm.invoke(prompt).content

            # Log the interaction for feedback purposes
            self._log_interaction(query, response, [doc for doc, _ in relevant_docs])

            return response
        except Exception as e:
            print(f"Error generating response: {e}")
            raise

    def simulate_feedback_loop(self, query: str, rounds: int = 3) -> None:
        """
        Simulate a feedback loop with multiple rounds of interaction.

        Args:
            query (str): The initial user query
            rounds (int): Number of feedback rounds to simulate (default: 3)
        """
        for round_num in range(1, rounds + 1):
            print(f"\n=== Feedback Round {round_num} ===")

            # Generate initial response
            response = self.generate_response(query)
            print(f"\nResponse:\n{response}")

            # Simulate user feedback (in a real system, this would come from actual users)
            if round_num == 1:
                # First round - assume partial satisfaction
                is_helpful = False
                feedback_text = (
                    "The answer was somewhat relevant but missed some key points."
                )
            elif round_num == 2:
                # Second round - improved but still not perfect
                is_helpful = True
                feedback_text = "Better, but could use more specific examples."
            else:
                # Third round - fully satisfied
                is_helpful = True
                feedback_text = "This answer completely addressed my question!"

            # Provide feedback
            self.provide_feedback(
                query=query,
                response=response,
                is_helpful=is_helpful,
                feedback_text=feedback_text,
            )

            print(
                f"\nFeedback provided: {'Helpful' if is_helpful else 'Not helpful'} - {feedback_text}"
            )

    def provide_feedback(
        self, query: str, response: str, is_helpful: bool, feedback_text: str = ""
    ) -> None:
        """
        Provide feedback on a generated response to improve the system.

        Args:
            query (str): The original user query
            response (str): The generated response
            is_helpful (bool): Whether the response was helpful
            feedback_text (str): Additional feedback comments
        """
        feedback_entry = {
            "timestamp": datetime.now().isoformat(),
            "query": query,
            "response": response,
            "is_helpful": is_helpful,
            "feedback_text": feedback_text,
            "action_taken": None,
        }

        if not is_helpful:
            # If response wasn't helpful, we might want to adjust the retrieval
            feedback_entry["action_taken"] = "Flagged for review"
            print("\nSystem note: This unhelpful response has been flagged for review.")

        self.feedback_log.append(feedback_entry)

    def _log_interaction(
        self, query: str, response: str, relevant_docs: List[Document]
    ) -> None:
        """
        Log an interaction for potential feedback analysis.

        Args:
            query (str): The user query
            response (str): Generated response
            relevant_docs (List[Document]): Documents retrieved for this query
        """
        interaction = {
            "timestamp": datetime.now().isoformat(),
            "query": query,
            "response": response,
            "retrieved_docs": [doc.to_dict() for doc in relevant_docs],
        }
        self.feedback_log.append(interaction)

    def analyze_feedback(self) -> Dict:
        """
        Analyze collected feedback to identify potential improvements.

        Returns:
            Dictionary containing analysis results
        """
        helpful_count = sum(
            1 for entry in self.feedback_log if entry.get("is_helpful", False)
        )
        unhelpful_count = sum(
            1
            for entry in self.feedback_log
            if "is_helpful" in entry and not entry["is_helpful"]
        )

        # Calculate average document relevance score from helpful responses
        relevance_scores = []
        for entry in self.feedback_log:
            if entry.get("is_helpful", False) and "retrieved_docs" in entry:
                # Count how many docs were actually used in the response
                context_used = sum(
                    1
                    for doc in entry["retrieved_docs"]
                    if doc["content"] in entry["response"]
                )
                relevance_scores.append(context_used / len(entry["retrieved_docs"]))

        avg_relevance = (
            sum(relevance_scores) / len(relevance_scores) if relevance_scores else 0
        )

        return {
            "total_interactions": len(self.feedback_log),
            "helpful_responses": helpful_count,
            "unhelpful_responses": unhelpful_count,
            "helpfulness_ratio": helpful_count
            / max(1, (helpful_count + unhelpful_count)),
            "average_relevance_score": avg_relevance,
        }

    def save_system(self, directory: str) -> None:
        """
        Save the RAG system's state to disk.

        Args:
            directory (str): Directory to save system files
        """
        try:
            os.makedirs(directory, exist_ok=True)

            # Save vector store
            self.vector_store.save_to_disk(os.path.join(directory, "vector_store.json"))

            # Save feedback log
            with open(os.path.join(directory, "feedback_log.json"), "w") as f:
                json.dump(self.feedback_log, f)
        except Exception as e:
            print(f"Error saving system: {e}")
            raise

    @classmethod
    def load_system(cls, directory: str, llm_model: str = "llama3.2:3b") -> "RAGSystem":
        """
        Load a RAG system from disk.

        Args:
            directory (str): Directory containing system files
            llm_model (str): Name of the Ollama model to use

        Returns:
            Loaded RAGSystem instance
        """
        try:
            system = cls(llm_model=llm_model)

            # Load vector store
            system.vector_store = VectorStore.load_from_disk(
                os.path.join(directory, "vector_store.json")
            )

            # Load feedback log
            with open(os.path.join(directory, "feedback_log.json"), "r") as f:
                system.feedback_log = json.load(f)

            return system
        except Exception as e:
            print(f"Error loading system: {e}")
            raise

In [ ]:
rag = RAGSystem()

In [ ]:
pdf_documents = ["./dataset/health supplements/1. dietary supplements - for whom.pdf"]

In [ ]:
rag.ingest_pdf_documents(pdf_documents)

In [ ]:
query = "How to lose weight?"

In [ ]:
rag.simulate_feedback_loop(query, rounds=3)

In [ ]:
analysis = rag.analyze_feedback()
print("\n=== Final Feedback Analysis ===")
print(f"Total interactions: {analysis['total_interactions']}")
print(f"Helpful responses: {analysis['helpful_responses']}")
print(f"Unhelpful responses: {analysis['unhelpful_responses']}")
print(f"Helpfulness ratio: {analysis['helpfulness_ratio']:.2f}")
print(f"Average relevance score: {analysis['average_relevance_score']:.2f}")

In [ ]:
rag.save_system("rag_system_state")